<a href="https://colab.research.google.com/github/arthursl12/POC1/blob/main/POC2_FD004_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
%pip install scikeras;
%pip install -U tensorflow-addons;

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import glob
import datetime

H:\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
H:\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
H:\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor

from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.base import BaseEstimator,RegressorMixin

In [4]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

from tensorflow.keras.metrics import RootMeanSquaredError as RMSE

from keras.callbacks import LambdaCallback
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.optimizers.schedules import ExponentialDecay

import tensorflow_addons as tfa
from tensorflow_addons.metrics import RSquare as R2



from scikeras.wrappers import KerasRegressor

In [5]:
sns.set_palette('colorblind')

In [6]:
# Reproducibility
seed = 42
os.environ['PYTHONHASHSEED']=str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [7]:
tf.config.experimental.enable_op_determinism()

# Data Preparation

In [8]:
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    IN_COLAB = True
    folder="/content/CMaps/"

    # Dataset Download 
    os.system('git clone https://github.com/arthursl12/dataset_2')
    os.system('mv /content/dataset_2/CMaps /content/CMaps')
    os.system('mv /content/dataset_2/data_processing /content/data_processing')
    os.system('rm -rf dataset_2')
else:
    print('Not running on CoLab')
    IN_COLAB = False
    folder="CMaps/"
    %cd dataset_2/

Not running on CoLab
C:\Users\Arthur Lima\POC\dataset_2


In [9]:
from data_processing.processing import DatasetProcessing
from data_processing.training import HyperparameterSearch, reclipper_scorer
from data_processing.eval import Evaluation

In [10]:
proc = DatasetProcessing()

## Data Integration

The data are provided as a zip-compressed text file with 26 columns of numbers, separated by spaces. Each row is a snapshot of data taken during a single operational cycle, each column is a different variable. The columns correspond to:  

1) unit number   
2) time, in cycles  
3) operational setting 1  
4) operational setting 2  
5) operational setting 3    
6) sensor measurement 1    
7) sensor measurement 2  
...  
26) sensor measurement 20


There are 6 conditions (or combinations) which the 3 operational settings can take.  
Condition 1: Altitude = 0, Mach Number = 0, TRA = 100  
Condition 2: Altitude = 10, Mach Number = 0.25, TRA = 100  
Condition 3: Altitude = 20, Mach Number = 0.7 TRA = 100  
Condition 4: Altitude = 25, Mach Number = 0.62, TRA = 60  
Condition 5: Altitude = 35 Mach Number = 0.84, TRA = 100  
Condition 6: Altitude = 42, Mach Number = 0.84, TRA = 100  
  
There is slight variation in all these conditions so you may get numbers like 24.453 instead of 25 exactly.

FD001: Condition 1 only  
FD002: Mix of all the conditions  
FD003: Condition 1 only  
FD004: Mix of all conditions  


In [11]:
index_cols, settings_cols, sensors_cols, cols = proc.column_names()
train, test, y_test = proc.read_dataset(4,folder=folder)
train

,unit_number,time,op_1,op_2,op_3,s_0,s_1,s_2,s_3,s_4,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,1,1,42.0049,0.8400,100.0,445.00,549.68,1343.43,1112.93,3.91,...,129.78,2387.99,8074.83,9.3335,0.02,330,2212,100.00,10.62,6.3670
1,1,2,20.0020,0.7002,100.0,491.19,606.07,1477.61,1237.50,9.35,...,312.59,2387.73,8046.13,9.1913,0.02,361,2324,100.00,24.37,14.6552
2,1,3,42.0038,0.8409,100.0,445.00,548.95,1343.12,1117.05,3.91,...,129.62,2387.97,8066.62,9.4007,0.02,329,2212,100.00,10.48,6.4213
3,1,4,42.0000,0.8400,100.0,445.00,548.70,1341.24,1118.03,3.91,...,129.80,2388.02,8076.05,9.3369,0.02,328,2212,100.00,10.54,6.4176
4,1,5,25.0063,0.6207,60.0,462.54,536.10,1255.23,1033.59,7.05,...,164.11,2028.08,7865.80,10.8366,0.02,305,1915,84.93,14.03,8.6754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61244,249,251,9.9998,0.2500,100.0,489.05,605.33,1516.36,1315.28,10.52,...,380.16,2388.73,8185.69,8.4541,0.03,372,2319,100.00,29.11,17.5234
61245,249,252,0.0028,0.0015,100.0,518.67,643.42,1598.92,1426.77,14.62,...,535.02,2388.46,8185.47,8.2221,0.03,396,2388,100.00,39.38,23.7151
61246,249,253,0.0029,0.0000,100.0,518.67,643.68,1607.72,1430.56,14.62,...,535.41,2388.48,8193.94,8.2525,0.03,395,2388,100.00,39.78,23.8270
61247,249,254,35.0046,0.8400,100.0,449.44,555.77,1381.29,1148.18,5.48,...,187.92,2388.83,8125.64,9.0515,0.02,337,2223,100.00,15.26,9.0774


## Preprocessing

### Test Set Transformation 
Test set has samples for all cycles, but has annotations only for last one

In [12]:
test.shape, y_test.shape

((41214, 26), (248, 1))

In [13]:
test_last = proc.transform_test_keep_setting(test)
test_last.head()

,op_1,op_2,op_3,s_0,s_1,s_2,s_3,s_4,s_5,s_6,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,25.0070,0.6214,60.0,462.54,537.66,1264.31,1046.41,7.05,8.99,176.56,...,166.19,2028.53,7890.31,10.7615,0.02,308,1915,84.93,14.41,8.6329
1,41.9989,0.8400,100.0,445.00,549.96,1354.05,1133.55,3.91,5.72,139.03,...,130.17,2387.72,8073.44,9.3925,0.02,331,2212,100.00,10.58,6.4325
2,42.0005,0.8401,100.0,445.00,549.47,1341.06,1118.90,3.91,5.69,139.26,...,130.73,2388.18,8095.58,9.2974,0.02,330,2212,100.00,10.61,6.3488
3,25.0018,0.6207,60.0,462.54,536.06,1253.49,1038.53,7.05,9.00,175.63,...,164.91,2028.30,7878.63,10.8396,0.02,306,1915,84.93,14.41,8.5696
4,25.0039,0.6200,60.0,462.54,537.36,1263.60,1052.52,7.05,9.03,175.53,...,164.95,2028.24,7873.75,10.9094,0.02,307,1915,84.93,14.19,8.6248


In [14]:
X_test = test_last

### Remaining Useful Life (RUL)

In [15]:
train = proc.add_remaining_useful_life_linear(train)
train[index_cols+['RUL']].head()

,unit_number,time,RUL
0,1,1,320
1,1,2,319
2,1,3,318
3,1,4,317
4,1,5,316


## Attributes and target separation

In [16]:
X_train, y_train = proc.X_y_train_divide_with_settings(train)

In [17]:
y_train.head()

,RUL
0,320
1,319
2,318
3,317
4,316


In [18]:
X_train.head()

,op_1,op_2,op_3,s_0,s_1,s_2,s_3,s_4,s_5,s_6,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,42.0049,0.8400,100.0,445.00,549.68,1343.43,1112.93,3.91,5.70,137.36,...,129.78,2387.99,8074.83,9.3335,0.02,330,2212,100.00,10.62,6.3670
1,20.0020,0.7002,100.0,491.19,606.07,1477.61,1237.50,9.35,13.61,332.10,...,312.59,2387.73,8046.13,9.1913,0.02,361,2324,100.00,24.37,14.6552
2,42.0038,0.8409,100.0,445.00,548.95,1343.12,1117.05,3.91,5.69,138.18,...,129.62,2387.97,8066.62,9.4007,0.02,329,2212,100.00,10.48,6.4213
3,42.0000,0.8400,100.0,445.00,548.70,1341.24,1118.03,3.91,5.70,137.98,...,129.80,2388.02,8076.05,9.3369,0.02,328,2212,100.00,10.54,6.4176
4,25.0063,0.6207,60.0,462.54,536.10,1255.23,1033.59,7.05,9.00,174.82,...,164.11,2028.08,7865.80,10.8366,0.02,305,1915,84.93,14.03,8.6754


## Training and Evaluation functions

In [19]:
eval = Evaluation()

In [20]:
search = HyperparameterSearch()

# MLP Construction

## Callbacks

In [21]:
# Early Stopping Callback
es = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                      patience=5, restore_best_weights=True)

In [22]:
# Printing Callback
def printLog(epoch, logs):
    print(
        f"E {epoch+1}\t: loss={logs['loss']:.3f}, "+
        f"rmse={logs['root_mean_squared_error']:.3f}, "+
        f"r2={logs['r_square']:.3f}; "+
        f"v_loss={logs['val_loss']:.3f}, "+
        f"v_rmse={logs['val_root_mean_squared_error']:.3f}, "+
        f"v_r2={logs['val_r_square']:.3f}; "
    )

printerCallback = LambdaCallback(on_epoch_end=printLog)

## Constants

In [23]:
# X_train must include indices
train3 = train.copy()
X_train_ = train3.drop(columns=[index_cols[1]]+["RUL"])

In [24]:
INPUT_SHAPE = 0

## Wrapper

In [25]:
from sys import is_finalizing
from sklearn.metrics import r2_score

class MLPWrapperRegressor(BaseEstimator,RegressorMixin):
    def __init__(self, basemodel=None, clip_y=-1,
                 include_settings=False, poly_degree=1,
                 scaler=StandardScaler()):
        # Base parameters
        self.basemodel = basemodel
        self.clip_y = clip_y
        self.poly_degree = poly_degree
        self.include_settings = include_settings

        # Column indexers
        self.feature_cols = sensors_cols
        if(include_settings):
            self.feature_cols = settings_cols + self.feature_cols
        self.base_feature_cols = self.feature_cols

        # Scaler and PolyFeatures transformers
        self.scaler = scaler
        self.polyft = PolynomialFeatures(degree=self.poly_degree, 
                                         include_bias=False)

    def clean_cols(self,df):
        if("unit_number" in df.columns): del df["unit_number"]
        if("unit_number" in df.columns): del df["unit_number"]
        if("time" in df.columns): del df["time"]
        if((not self.include_settings)): 
            for col in settings_cols:
                if(col in df.columns): del df[col]
        return df

    def fit(self, X=None, y=None):
        # Drop irrelevant column
        data = pd.DataFrame(X).copy()
        data = self.clean_cols(data)
        
        # Apply polynomial features and add them to the dataframe
        transf = self.polyft.fit_transform(data)
        data = pd.DataFrame(transf, columns=
                              self.polyft.get_feature_names_out())
        self.feature_cols = list(self.polyft.get_feature_names_out())

        # Scale the data
        data = self.scaler.fit_transform(data)
        data = pd.DataFrame(data, 
                            columns=self.scaler.get_feature_names_out())
        X_train = data.copy()
        
        # Clip and transform labels
        data2 = pd.DataFrame(y, columns=["RUL"]).copy()
        if (self.clip_y > 0):
            data2["RUL"].clip(upper=self.clip_y, inplace=True)
        y_train = data2

        # Update input shape for future use
        global INPUT_SHAPE
        INPUT_SHAPE = X_train.shape[1]

        # Fit model
        self.basemodel.fit(X_train,y_train)
        return self
        
    def predict(self, X=None):
        # Perform transformation, if not done
        if (len(X.columns) != len(self.feature_cols)):
            X_train = self.transform_features(X)
        else:
            X_train = X
        return self.basemodel.predict(X_train)

    def transform_features(self, df):
        # Drop irrelevant column
        data = pd.DataFrame(df).copy()
        data = self.clean_cols(data)
        
        # Apply polynomial features and add them to the dataframe
        transf = self.polyft.transform(data)
        data = pd.DataFrame(transf, columns=
                              self.polyft.get_feature_names_out())
        # self.feature_cols = list(self.polyft.get_feature_names_out())

        # Scale the data
        data = self.scaler.transform(data)
        data = pd.DataFrame(data, 
                            columns=self.scaler.get_feature_names_out())
        return data

    def score(self, X, y, sample_weight=None):
        X_test = self.transform_features(X)

        # Clip and transform labels
        data2 = pd.DataFrame(y, columns=["RUL"]).copy()
        if (self.clip_y > 0):
            data2["RUL"].clip(upper=self.clip_y, inplace=True)
        y_test = data2

        # Predict on test data
        y_pred = self.predict(X_test)
        return r2_score(y_test, y_pred, sample_weight=sample_weight)

## Test Data

In [26]:
def scale_test(test,model):
    test2 = pd.DataFrame(test)
    test2 = model.clean_cols(test2)

    # Apply polynomial features
    transf = model.polyft.transform(test2)
    test2 = pd.DataFrame(transf, 
                         columns=model.polyft.get_feature_names_out())

    # Scale the data (with train data parameters)
    test2 = model.scaler.transform(test2)
    test2 = pd.DataFrame(test2, 
                         columns=model.polyft.get_feature_names_out())
    return test2

## Constructor

In [27]:
def create_model(optim=Adam, learning_rate=1e-3, 
                 layer1=32  , activation1="tanh"    , dropout1=0.1,
                 layer2=None, activation2="tanh"    , dropout2=0.1,
                 layer3=None, activation3="tanh"    , dropout3=0.1,
                 layer4=None, activation4="tanh"    , dropout4=0.1,
                 print_summary=False, loss='mean_squared_error',
                 metrics=[tf.keras.metrics.MeanSquaredError()]):
    model = Sequential()

    # Fully Connected Layer
    model.add(Dense(layer1, input_dim=INPUT_SHAPE, activation=activation1))
    model.add(Dropout(dropout1))

    # Additional hidden layers
    if(layer2 is not None):
        model.add(Dense(layer2, activation=activation2))
        model.add(Dropout(dropout2))
        if (layer3 is not None):
            model.add(Dense(layer3, activation=activation3))
            model.add(Dropout(dropout3))
            if (layer4 is not None):
                model.add(Dense(layer4, activation=activation4))
                model.add(Dropout(dropout4))

    # Output Layer
    model.add(Dense(1))

    model.compile(loss=loss, optimizer=optim(learning_rate=learning_rate), 
                  metrics=metrics)
    
    if(print_summary): model.summary()
    return model

# MLP-1

## Linear RUL 

Score: 0.540842574601971  
Test: 0.
```
('basemodel__batch_size', 275),
('basemodel__epochs', 37),
('basemodel__model__activation1', 'relu'),
('basemodel__model__dropout1', 0.2142001322493293),
('basemodel__model__layer1', 512),
('basemodel__model__learning_rate', 0.007899917433901438),
('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.1),
('include_settings', True),
('scaler', StandardScaler())
```


In [28]:
CLIP=-1

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=1,
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=275,
                           epochs=37,
                           model__activation1='relu',
                           model__dropout1=0.2142001322493293, 
                           model__layer1=512, 
                           model__learning_rate=0.007899917433901438,
                           model__optim=Adam,
                           validation_split=0.1, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=275, callbacks=[<keras.callbacks.EarlyStopping object at 0x00000281BF12DF40>, <keras.callbacks.LambdaCallback object at 0x00000281BF13F100>], epochs=37, model=<function create_model at 0x00000281C1861DC0>, model__activation1='relu', model__dropout1=0.2142001322493293, model__layer1=512, model__learning_rate=0.007899917433901438, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x00000281BF13FC70>, <tensorflow_addons.metrics.r_square.RSquare object at 0x00000281C1996610>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.1, verbose=0),
                    include_settings=True)

In [29]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               12800     
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 513       
                                                                 
Total params: 13,313
Trainable params: 13,313
Non-trainable params: 0
_________________________________________________________________
E 1	: loss=9231.953, rmse=96.083, r2=-0.127; v_loss=5201.883, v_rmse=72.124, v_r2=0.244; 
E 2	: loss=5315.319, rmse=72.906, r2=0.351; v_loss=3927.683, v_rmse=62.671, v_r2=0.429; 
E 3	: loss=4330.736, rmse=65.808, r2=0.471; v_loss=3549.014, v_rmse=59.574, v_r2=0.484; 
E 4	: loss=4053.428, rmse=63.667, r2=0.505; v_lo

In [30]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.154,RMSE=-50.149
Finished: 2022-11-03 08:35:57.871934


## Non-Linear RUL

Score: 0.8250779110568915  
Test: 0.
```
('basemodel__batch_size', 32),
('basemodel__epochs', 49),
('basemodel__model__activation1', 'tanh'),
('basemodel__model__dropout1', 0.1),
('basemodel__model__layer1', 512),
('basemodel__model__learning_rate', 0.01),
('basemodel__model__optim', keras.optimizer_v2.rmsprop.RMSprop),
('basemodel__validation_split', 0.14787301913541884),
('clip_y', 80),
('include_settings', True),
('scaler', StandardScaler())
```

In [31]:
CLIP=80

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=1,
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=32,
                           epochs=49,
                           model__activation1='tanh',
                           model__dropout1=0.1, 
                           model__layer1=512, 
                           model__learning_rate=0.01,
                           model__optim=RMSprop,
                           validation_split=0.14787301913541884, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=32, callbacks=[<keras.callbacks.EarlyStopping object at 0x00000281BF12DF40>, <keras.callbacks.LambdaCallback object at 0x00000281BF13F100>], epochs=49, model=<function create_model at 0x00000281C1861DC0>, model__activation1='tanh', model__dropout1=0.1, model__layer1=512, model__learning_rate=0.01, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x00000281D7DB2550>, <tensorflow_addons.metrics.r_square.RSquare object at 0x00000281BF079EB0>], model__optim=<class 'keras.optimizer_v2.rmsprop.RMSprop'>, print_summary=True, validation_split=0.14787301913541884, verbose=0),
                    clip_y=80, include_settings=True)

In [32]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 512)               12800     
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 513       
                                                                 
Total params: 13,313
Trainable params: 13,313
Non-trainable params: 0
_________________________________________________________________
E 1	: loss=338.300, rmse=18.393, r2=0.363; v_loss=184.123, v_rmse=13.569, v_r2=0.662; 
E 2	: loss=204.642, rmse=14.305, r2=0.615; v_loss=140.924, v_rmse=11.871, v_r2=0.741; 
E 3	: loss=172.777, rmse=13.144, r2=0.675; v_loss=126.212, v_rmse=11.234, v_r2=0.768; 
E 4	: loss=156.098, rmse=12.494, r2=0.706; v_loss=111

In [33]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.834,RMSE=-11.002
Finished: 2022-11-03 08:45:06.828842


## PolyFeatures + Linear RUL 


Score: 0.5269773808976338  
Test: 0.
```
('basemodel__batch_size', 315),
('basemodel__epochs', 38),
('basemodel__model__activation1', 'relu'),
('basemodel__model__dropout1', 0.38326321899773097),
('basemodel__model__layer1', 350),
('basemodel__model__learning_rate', 0.0024970992234534897),
('basemodel__model__optim', keras.optimizer_v2.rmsprop.RMSprop),
('basemodel__validation_split', 0.23038975772370296),
('include_settings', True),
('poly_degree', 2),
('scaler', StandardScaler())
```


In [34]:
CLIP=-1

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=2,
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=315,
                           epochs=38,
                           model__activation1='relu',
                           model__dropout1=0.38326321899773097, 
                           model__layer1=350, 
                           model__learning_rate=0.0024970992234534897,
                           model__optim=RMSprop,
                           validation_split=0.23038975772370296, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=315, callbacks=[<keras.callbacks.EarlyStopping object at 0x00000281BF12DF40>, <keras.callbacks.LambdaCallback object at 0x00000281BF13F100>], epochs=38, model=<function create_model at 0x00000281C1861DC0>, model__activation1='relu', model__dropout1=0.38326321899773097, model__layer1=350, model__learning_rate=0.0024970992234534897, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x00000282F5643070>, <tensorflow_addons.metrics.r_square.RSquare object at 0x00000282F56432E0>], model__optim=<class 'keras.optimizer_v2.rmsprop.RMSprop'>, print_summary=True, validation_split=0.23038975772370296, verbose=0),
                    include_settings=True, poly_degree=2)

In [35]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 350)               113750    
                                                                 
 dropout_2 (Dropout)         (None, 350)               0         
                                                                 
 dense_5 (Dense)             (None, 1)                 351       
                                                                 
Total params: 114,101
Trainable params: 114,101
Non-trainable params: 0
_________________________________________________________________
E 1	: loss=9158.690, rmse=95.701, r2=-0.094; v_loss=6166.658, v_rmse=78.528, v_r2=0.112; 
E 2	: loss=6969.643, rmse=83.484, r2=0.168; v_loss=5068.129, v_rmse=71.191, v_r2=0.270; 
E 3	: loss=6014.202, rmse=77.551, r2=0.282; v_loss=5139.114, v_rmse=71.688, v_r2=0.260; 
E 4	: loss=5373.119, rmse=73.302, r2=0.358; 

In [36]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.320,RMSE=-44.954
Finished: 2022-11-03 08:47:03.111816


## PolyFeatures + Non-Linear RUL


Score: 0.7360051567025246  
Test: 0.
```
('basemodel__batch_size', 318),
('basemodel__epochs', 37),
('basemodel__model__activation1', 'elu'),
('basemodel__model__dropout1', 0.1),
('basemodel__model__layer1', 31),
('basemodel__model__learning_rate', 0.0064651627718996674),
('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.1),
('clip_y', 93),
('include_settings', True),
('poly_degree', 2),
('scaler', StandardScaler())
```

In [37]:
CLIP=93

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=2,
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=318,
                           epochs=37,
                           model__activation1='elu',
                           model__dropout1=0.1, 
                           model__layer1=31, 
                           model__learning_rate=0.0064651627718996674,
                           model__optim=Adam,
                           validation_split=0.1, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=318, callbacks=[<keras.callbacks.EarlyStopping object at 0x00000281BF12DF40>, <keras.callbacks.LambdaCallback object at 0x00000281BF13F100>], epochs=37, model=<function create_model at 0x00000281C1861DC0>, model__activation1='elu', model__dropout1=0.1, model__layer1=31, model__learning_rate=0.0064651627718996674, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x00000282F5A48B20>, <tensorflow_addons.metrics.r_square.RSquare object at 0x00000282F580B460>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.1, verbose=0),
                    clip_y=93, include_settings=True, poly_degree=2)

In [38]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 31)                10075     
                                                                 
 dropout_3 (Dropout)         (None, 31)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                 32        
                                                                 
Total params: 10,107
Trainable params: 10,107
Non-trainable params: 0
_________________________________________________________________
E 1	: loss=1210.043, rmse=34.786, r2=-0.528; v_loss=578.250, v_rmse=24.047, v_r2=0.272; 
E 2	: loss=580.147, rmse=24.086, r2=0.267; v_loss=370.215, v_rmse=19.241, v_r2=0.534; 
E 3	: loss=369.801, rmse=19.230, r2=0.533; v_loss=243.712, v_rmse=15.611, v_r2=0.693; 
E 4	: loss=309.925, rmse=17.605, r2=0.609; v_loss=1

In [39]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.771,RMSE=-15.395
Finished: 2022-11-03 08:48:40.293557


# MLP-2 

## Linear RUL

Score: 0.5434403412387998  
Test: 0.
```
('basemodel__batch_size', 475),
('basemodel__epochs', 31),
('basemodel__model__activation1', 'selu'),
('basemodel__model__activation2', 'sigmoid'),
('basemodel__model__dropout1', 0.1551853147879086),
('basemodel__model__dropout2', 0.34328552130273915),
('basemodel__model__layer1', 115),
('basemodel__model__layer2', 512),
('basemodel__model__learning_rate', 0.006720220722226725),
('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.29870359291003556),
('include_settings', True),
('scaler', StandardScaler())
```

In [40]:
CLIP=-1

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=1,
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=475,
                           epochs=31,
                           model__activation1='selu',
                           model__activation2='sigmoid',
                           model__dropout1=0.1551853147879086, 
                           model__dropout2=0.34328552130273915, 
                           model__layer1=115, 
                           model__layer2=512, 
                           model__learning_rate=0.006720220722226725,
                           model__optim=Adam,
                           validation_split=0.29870359291003556, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=475, callbacks=[<keras.callbacks.EarlyStopping object at 0x00000281BF12DF40>, <keras.callbacks.LambdaCallback object at 0x00000281BF13F100>], epochs=31, model=<function create_model at 0x00000281C1861DC0>, model__activation1='selu', model__activation2='sigmoid', model__dropout1=0.1551853147879086, model__dropo...odel__learning_rate=0.006720220722226725, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x00000282F5D3B130>, <tensorflow_addons.metrics.r_square.RSquare object at 0x00000281CA839640>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.29870359291003556, verbose=0),
                    include_settings=True)

In [41]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 115)               2875      
                                                                 
 dropout_4 (Dropout)         (None, 115)               0         
                                                                 
 dense_9 (Dense)             (None, 512)               59392     
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_10 (Dense)            (None, 1)                 513       
                                                                 
Total params: 62,780
Trainable params: 62,780
Non-trainable params: 0
_________________________________________________________________
E 1	: loss=11412.845, rmse=106.831, r2=-0.391; v_l

In [42]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.244,RMSE=-47.413
Finished: 2022-11-03 08:50:00.179244


## Non-Linear RUL

Score: 0.8092955367135408  
Test: 0.
```
('basemodel__batch_size', 190),
('basemodel__epochs', 50),
('basemodel__model__activation1', 'elu'),
('basemodel__model__activation2', 'elu'),
('basemodel__model__dropout1', 0.10508709834130409),
('basemodel__model__dropout2', 0.15866472852550143),
('basemodel__model__layer1', 130),
('basemodel__model__layer2', 512),
('basemodel__model__learning_rate', 0.01),
('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.5585747596345988),
('clip_y', 80),
('include_settings', True),
('scaler', StandardScaler())
```

In [43]:
CLIP=80

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=1,
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=190,
                           epochs=50,
                           model__activation1='elu',
                           model__activation2='elu',
                           model__dropout1=0.10508709834130409, 
                           model__dropout2=0.15866472852550143, 
                           model__layer1=130, 
                           model__layer2=512, 
                           model__learning_rate=0.01,
                           model__optim=Adam,
                           validation_split=0.5585747596345988, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=190, callbacks=[<keras.callbacks.EarlyStopping object at 0x00000281BF12DF40>, <keras.callbacks.LambdaCallback object at 0x00000281BF13F100>], epochs=50, model=<function create_model at 0x00000281C1861DC0>, model__activation1='elu', model__activation2='elu', model__dropout1=0.10508709834130409, model__dropout2=...2=512, model__learning_rate=0.01, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x00000281CA774430>, <tensorflow_addons.metrics.r_square.RSquare object at 0x00000282F5D2FAC0>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.5585747596345988, verbose=0),
                    clip_y=80, include_settings=True)

In [44]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 130)               3250      
                                                                 
 dropout_6 (Dropout)         (None, 130)               0         
                                                                 
 dense_12 (Dense)            (None, 512)               67072     
                                                                 
 dropout_7 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 1)                 513       
                                                                 
Total params: 70,835
Trainable params: 70,835
Non-trainable params: 0
_________________________________________________________________
E 1	: loss=593.733, rmse=24.367, r2=-0.113; v_loss

In [45]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.825,RMSE=-11.318
Finished: 2022-11-03 08:51:25.841779


## PolyFeatures + Linear RUL

Score: 0.5403727057267015  
Test: 0.
```
('basemodel__batch_size', 123),
('basemodel__epochs', 50),
('basemodel__model__activation1', 'selu'),
('basemodel__model__activation2', 'selu'),
('basemodel__model__dropout1', 0.2440452992785985),
('basemodel__model__dropout2', 0.7488072091140527),
('basemodel__model__layer1', 289),
('basemodel__model__layer2', 490),
('basemodel__model__learning_rate', 0.0002194806594294514),
('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.2571693518261989),
('include_settings', True),
('poly_degree', 2),
('scaler', StandardScaler())
```

In [46]:
CLIP=-1

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=2,
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=123,
                           epochs=50,
                           model__activation1='selu',
                           model__activation2='selu',
                           model__dropout1=0.2440452992785985, 
                           model__dropout2=0.7488072091140527, 
                           model__layer1=289, 
                           model__layer2=490, 
                           model__learning_rate=0.0002194806594294514,
                           model__optim=Adam,
                           validation_split=0.2571693518261989, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=123, callbacks=[<keras.callbacks.EarlyStopping object at 0x00000281BF12DF40>, <keras.callbacks.LambdaCallback object at 0x00000281BF13F100>], epochs=50, model=<function create_model at 0x00000281C1861DC0>, model__activation1='selu', model__activation2='selu', model__dropout1=0.2440452992785985, model__dropout2..._rate=0.0002194806594294514, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x00000282F61CE730>, <tensorflow_addons.metrics.r_square.RSquare object at 0x00000282F5DAF6A0>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.2571693518261989, verbose=0),
                    include_settings=True, poly_degree=2)

In [47]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 289)               93925     
                                                                 
 dropout_8 (Dropout)         (None, 289)               0         
                                                                 
 dense_15 (Dense)            (None, 490)               142100    
                                                                 
 dropout_9 (Dropout)         (None, 490)               0         
                                                                 
 dense_16 (Dense)            (None, 1)                 491       
                                                                 
Total params: 236,516
Trainable params: 236,516
Non-trainable params: 0
_________________________________________________________________
E 1	: loss=11667.834, rmse=108.018, r2=-0.395; v

In [48]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.241,RMSE=-47.491
Finished: 2022-11-03 08:53:36.605466


## PolyFeatures + Non-Linear RUL


Score: 0.8137440377581179  
Test: 0.
```
('basemodel__batch_size', 330),
('basemodel__epochs', 28),
('basemodel__model__activation1', 'selu'),
('basemodel__model__activation2', 'sigmoid'),
('basemodel__model__dropout1', 0.35104184791262427),
('basemodel__model__dropout2', 0.13083828310831028),
('basemodel__model__layer1', 340),
('basemodel__model__layer2', 191),
('basemodel__model__learning_rate', 0.009916920939867237),
('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.1),
('clip_y', 87),
('include_settings', True),
('poly_degree', 2),
('scaler', StandardScaler())
```

In [58]:
CLIP=87

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=MinMaxScaler(), poly_degree=2,
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=163,
                           epochs=28,
                           model__activation1='selu',
                           model__activation2='relu',
                           model__dropout1=0.1203636705927238, 
                           model__dropout2=0.3108545284702169, 
                           model__layer1=18, 
                           model__layer2=364, 
                           model__learning_rate=0.009076852273996898,
                           model__optim=Adam,
                           validation_split=0.14877733495106543, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=163, callbacks=[<keras.callbacks.EarlyStopping object at 0x00000281BF12DF40>, <keras.callbacks.LambdaCallback object at 0x00000281BF13F100>], epochs=28, model=<function create_model at 0x00000281C1861DC0>, model__activation1='selu', model__activation2='relu', model__dropout1=0.1203636705927238, model__dropout2...__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x00000282F3D84670>, <tensorflow_addons.metrics.r_square.RSquare object at 0x00000282F3D76EE0>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.14877733495106543, verbose=0),
                    clip_y=87, include_settings=True, poly_degree=2,
                    scaler=MinMaxScaler())

In [59]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_26 (Dense)            (None, 18)                5850      
                                                                 
 dropout_16 (Dropout)        (None, 18)                0         
                                                                 
 dense_27 (Dense)            (None, 364)               6916      
                                                                 
 dropout_17 (Dropout)        (None, 364)               0         
                                                                 
 dense_28 (Dense)            (None, 1)                 365       
                                                                 
Total params: 13,131
Trainable params: 13,131
Non-trainable params: 0
_________________________________________________________________
E 1	: loss=797.643, rmse=28.243, r2=-0.202; v_los

In [60]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.573,RMSE=-19.508
Finished: 2022-11-03 09:01:04.753720


# MLP-3 

## Linear RUL

Score: 0.5387760282104624    
Test: 0.
```
('basemodel__batch_size', 314),
('basemodel__epochs', 28),
('basemodel__model__activation1', 'tanh'),
('basemodel__model__activation2', 'selu'),
('basemodel__model__activation3', 'selu'),
('basemodel__model__dropout1', 0.23024902825797536),
('basemodel__model__dropout2', 0.27996962001171605),
('basemodel__model__dropout3', 0.6837395507589162),
('basemodel__model__layer1', 288),
('basemodel__model__layer2', 158),
('basemodel__model__layer3', 381),
('basemodel__model__learning_rate', 0.002110763064656101),
('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.5017895339783691),
('include_settings', True),
('scaler', MinMaxScaler())
```

In [61]:
CLIP=-1

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=MinMaxScaler(), poly_degree=1,
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=314,
                           epochs=28,
                           model__activation1='tanh',
                           model__activation2='selu',
                           model__activation3='selu',
                           model__dropout1=0.23024902825797536, 
                           model__dropout2=0.27996962001171605, 
                           model__dropout3=0.6837395507589162, 
                           model__layer1=288, 
                           model__layer2=158, 
                           model__layer3=381, 
                           model__learning_rate=0.002110763064656101,
                           model__optim=Adam,
                           validation_split=0.5017895339783691, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=314, callbacks=[<keras.callbacks.EarlyStopping object at 0x00000281BF12DF40>, <keras.callbacks.LambdaCallback object at 0x00000281BF13F100>], epochs=28, model=<function create_model at 0x00000281C1861DC0>, model__activation1='tanh', model__activation2='selu', model__activation3='selu', model__dropout1=0.230249....002110763064656101, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x00000282F47B5A30>, <tensorflow_addons.metrics.r_square.RSquare object at 0x00000282F3DDE0D0>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.5017895339783691, verbose=0),
                    include_settings=True, scaler=MinMaxScaler())

In [62]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_29 (Dense)            (None, 288)               7200      
                                                                 
 dropout_18 (Dropout)        (None, 288)               0         
                                                                 
 dense_30 (Dense)            (None, 158)               45662     
                                                                 
 dropout_19 (Dropout)        (None, 158)               0         
                                                                 
 dense_31 (Dense)            (None, 381)               60579     
                                                                 
 dropout_20 (Dropout)        (None, 381)               0         
                                                                 
 dense_32 (Dense)            (None, 1)               

In [63]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=-0.243,RMSE=-60.796
Finished: 2022-11-03 09:03:03.565239


## Non-Linear RUL

Score: 0.8191894908542713    
Test: 0.
```
('basemodel__batch_size', 277),
('basemodel__epochs', 50),
('basemodel__model__activation1', 'tanh'),
('basemodel__model__activation2', 'selu'),
('basemodel__model__activation3', 'relu'),
('basemodel__model__dropout1', 0.15180815010798232),
('basemodel__model__dropout2', 0.4099007005621048),
('basemodel__model__dropout3', 0.1),
('basemodel__model__layer1', 197),
('basemodel__model__layer2', 353),
('basemodel__model__layer3', 261),
('basemodel__model__learning_rate', 0.0041467234962615715),
('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.17508243717937616),
('clip_y', 80),
('include_settings', True),
('scaler', StandardScaler())
```

In [64]:
CLIP=80

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=1,
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=277,
                           epochs=50,
                           model__activation1='tanh',
                           model__activation2='selu',
                           model__activation3='relu',
                           model__dropout1=0.15180815010798232, 
                           model__dropout2=0.4099007005621048, 
                           model__dropout3=0.1, 
                           model__layer1=197, 
                           model__layer2=353, 
                           model__layer3=261,  
                           model__learning_rate=0.0041467234962615715,
                           model__optim=Adam,
                           validation_split=0.17508243717937616, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=277, callbacks=[<keras.callbacks.EarlyStopping object at 0x00000281BF12DF40>, <keras.callbacks.LambdaCallback object at 0x00000281BF13F100>], epochs=50, model=<function create_model at 0x00000281C1861DC0>, model__activation1='tanh', model__activation2='selu', model__activation3='relu', model__dropout1=0.151808...ing_rate=0.0041467234962615715, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x00000282F4963D30>, <tensorflow_addons.metrics.r_square.RSquare object at 0x00000282F4963F40>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.17508243717937616, verbose=0),
                    clip_y=80, include_settings=True)

In [65]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_33 (Dense)            (None, 197)               4925      
                                                                 
 dropout_21 (Dropout)        (None, 197)               0         
                                                                 
 dense_34 (Dense)            (None, 353)               69894     
                                                                 
 dropout_22 (Dropout)        (None, 353)               0         
                                                                 
 dense_35 (Dense)            (None, 261)               92394     
                                                                 
 dropout_23 (Dropout)        (None, 261)               0         
                                                                 
 dense_36 (Dense)            (None, 1)               

In [66]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.810,RMSE=-11.799
Finished: 2022-11-03 09:24:51.765932


## PolyFeatures + Linear RUL

Score: 0.5434197290446054    
Test: 0.
```
('basemodel__batch_size', 254),
('basemodel__epochs', 44),
('basemodel__model__activation1', 'relu'),
('basemodel__model__activation2', 'selu'),
('basemodel__model__activation3', 'elu'),
('basemodel__model__dropout1', 0.22005754217690313),
('basemodel__model__dropout2', 0.2204345848078386),
('basemodel__model__dropout3', 0.6505425209498465),
('basemodel__model__layer1', 165),
('basemodel__model__layer2', 512),
('basemodel__model__layer3', 485),
('basemodel__model__learning_rate', 0.0023431044122668105),
('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.1),
('include_settings', True),
('poly_degree', 3),
('scaler', StandardScaler())
```


In [67]:
CLIP=-1

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=3,
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=254,
                           epochs=44,
                           model__activation1='relu',
                           model__activation2='selu',
                           model__activation3='elu',
                           model__dropout1=0.22005754217690313, 
                           model__dropout2=0.2204345848078386, 
                           model__dropout3=0.6505425209498465, 
                           model__layer1=165, 
                           model__layer2=512, 
                           model__layer3=485, 
                           model__learning_rate=0.0023431044122668105,
                           model__optim=Adam,
                           validation_split=0.1, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=254, callbacks=[<keras.callbacks.EarlyStopping object at 0x00000281BF12DF40>, <keras.callbacks.LambdaCallback object at 0x00000281BF13F100>], epochs=44, model=<function create_model at 0x00000281C1861DC0>, model__activation1='relu', model__activation2='selu', model__activation3='elu', model__dropout1=0.2200575...model__learning_rate=0.0023431044122668105, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x0000028301B9EBE0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x0000028301B9EE50>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.1, verbose=0),
                    include_settings=True, poly_degree=3)

In [68]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_37 (Dense)            (None, 165)               482625    
                                                                 
 dropout_24 (Dropout)        (None, 165)               0         
                                                                 
 dense_38 (Dense)            (None, 512)               84992     
                                                                 
 dropout_25 (Dropout)        (None, 512)               0         
                                                                 
 dense_39 (Dense)            (None, 485)               248805    
                                                                 
 dropout_26 (Dropout)        (None, 485)               0         
                                                                 
 dense_40 (Dense)            (None, 1)               

In [69]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=-0.848,RMSE=-74.113
Finished: 2022-11-03 09:27:27.482022


## PolyFeatures + Non-Linear RUL


Score: 0.8120876882657063   
Test: 0.
```
('basemodel__batch_size', 127),
('basemodel__epochs', 32),
('basemodel__model__activation1', 'tanh'),
('basemodel__model__activation2', 'sigmoid'),
('basemodel__model__activation3', 'selu'),
('basemodel__model__dropout1', 0.3239516474706593),
('basemodel__model__dropout2', 0.4398002541265835),
('basemodel__model__dropout3', 0.11820142028402897),
('basemodel__model__layer1', 300),
('basemodel__model__layer2', 387),
('basemodel__model__layer3', 97),
('basemodel__model__learning_rate', 0.0007470235779685965),
('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.27478903553784617),
('clip_y', 88),
('include_settings', True),
('poly_degree', 2),
('scaler', StandardScaler())
```

In [70]:
CLIP=88

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=2,
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=127,
                           epochs=32,
                           model__activation1='tanh',
                           model__activation2='sigmoid',
                           model__activation3='selu',
                           model__dropout1=0.3239516474706593, 
                           model__dropout2=0.4398002541265835, 
                           model__dropout3=0.11820142028402897, 
                           model__layer1=300, 
                           model__layer2=387, 
                           model__layer3=97, 
                           model__learning_rate=0.0007470235779685965,
                           model__optim=Adam,
                           validation_split=0.27478903553784617, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=127, callbacks=[<keras.callbacks.EarlyStopping object at 0x00000281BF12DF40>, <keras.callbacks.LambdaCallback object at 0x00000281BF13F100>], epochs=32, model=<function create_model at 0x00000281C1861DC0>, model__activation1='tanh', model__activation2='sigmoid', model__activation3='selu', model__dropout1=0.323...7470235779685965, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x00000281D7D86B20>, <tensorflow_addons.metrics.r_square.RSquare object at 0x00000281D7D866A0>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.27478903553784617, verbose=0),
                    clip_y=88, include_settings=True, poly_degree=2)

In [71]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_41 (Dense)            (None, 300)               97500     
                                                                 
 dropout_27 (Dropout)        (None, 300)               0         
                                                                 
 dense_42 (Dense)            (None, 387)               116487    
                                                                 
 dropout_28 (Dropout)        (None, 387)               0         
                                                                 
 dense_43 (Dense)            (None, 97)                37636     
                                                                 
 dropout_29 (Dropout)        (None, 97)                0         
                                                                 
 dense_44 (Dense)            (None, 1)               

In [72]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.822,RMSE=-12.765
Finished: 2022-11-03 09:29:17.819282


# MLP-4 

## Linear RUL


Score: 0.542517835612285    
Test: 0.
```
('basemodel__batch_size', 239),
('basemodel__epochs', 44),
('basemodel__model__activation1', 'relu'),
('basemodel__model__activation2', 'sigmoid'),
('basemodel__model__activation3', 'selu'),
('basemodel__model__activation4', 'selu'),
('basemodel__model__dropout1', 0.27228148001908253),
('basemodel__model__dropout2', 0.7227228862335966),
('basemodel__model__dropout3', 0.8720098451745675),
('basemodel__model__dropout4', 0.23520720034744447),
('basemodel__model__layer1', 222),
('basemodel__model__layer2', 355),
('basemodel__model__layer3', 254),
('basemodel__model__layer4', 94),
('basemodel__model__learning_rate', 0.006241072056393434),
('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.3776498733118778),
('include_settings', True),
('scaler', StandardScaler())
```


In [73]:
CLIP=-1

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=1,
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=239,
                           epochs=44,
                           model__activation1='relu',
                           model__activation2='sigmoid',
                           model__activation3='selu',
                           model__activation4='selu',
                           model__dropout1=0.27228148001908253, 
                           model__dropout2=0.7227228862335966, 
                           model__dropout3=0.8720098451745675, 
                           model__dropout4=0.23520720034744447, 
                           model__layer1=222, 
                           model__layer2=355, 
                           model__layer3=254, 
                           model__layer4=94, 
                           model__learning_rate=0.006241072056393434,
                           model__optim=Adam,
                           validation_split=0.3776498733118778, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=239, callbacks=[<keras.callbacks.EarlyStopping object at 0x00000281BF12DF40>, <keras.callbacks.LambdaCallback object at 0x00000281BF13F100>], epochs=44, model=<function create_model at 0x00000281C1861DC0>, model__activation1='relu', model__activation2='sigmoid', model__activation3='selu', model__activation4='s...model__learning_rate=0.006241072056393434, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x00000282F561E8B0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x00000282F5C6EDF0>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.3776498733118778, verbose=0),
                    include_settings=True)

In [74]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_45 (Dense)            (None, 222)               5550      
                                                                 
 dropout_30 (Dropout)        (None, 222)               0         
                                                                 
 dense_46 (Dense)            (None, 355)               79165     
                                                                 
 dropout_31 (Dropout)        (None, 355)               0         
                                                                 
 dense_47 (Dense)            (None, 254)               90424     
                                                                 
 dropout_32 (Dropout)        (None, 254)               0         
                                                                 
 dense_48 (Dense)            (None, 94)              

In [75]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.294,RMSE=-45.809
Finished: 2022-11-03 09:30:59.787085


## Non-Linear RUL

Score: 0.7979100870641017    
Test: 0.
```
('basemodel__batch_size', 125),
('basemodel__epochs', 40),
('basemodel__model__activation1', 'tanh'),
('basemodel__model__activation2', 'tanh'),
('basemodel__model__activation3', 'relu'),
('basemodel__model__activation4', 'selu'),
('basemodel__model__dropout1', 0.4239282276871049),
('basemodel__model__dropout2', 0.7682276844296427),
('basemodel__model__dropout3', 0.37470248079723845),
('basemodel__model__dropout4', 0.5586631760668447),
('basemodel__model__layer1', 486),
('basemodel__model__layer2', 470),
('basemodel__model__layer3', 244),
('basemodel__model__layer4', 123),
('basemodel__model__learning_rate', 0.0006832900387850664),
('basemodel__model__optim', keras.optimizer_v2.rmsprop.RMSprop),
('basemodel__validation_split', 0.2233573353320119),
('clip_y', 80),
('include_settings', True),
('scaler', StandardScaler())
```


In [85]:
CLIP=125

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=1,
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=125,
                           epochs=250,
                           model__activation1='tanh',
                           model__activation2='tanh',
                           model__activation3='relu',
                           model__activation4='selu',
                           model__dropout1=0.4239282276871049, 
                           model__dropout2=0.7682276844296427, 
                           model__dropout3=0.37470248079723845, 
                           model__dropout4=0.5586631760668447, 
                           model__layer1=486, 
                           model__layer2=470, 
                           model__layer3=244, 
                           model__layer4=123, 
                           model__learning_rate=0.0006832900387850664,
                           model__optim=RMSprop,
                           validation_split=0.2233573353320119, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=125, callbacks=[<keras.callbacks.EarlyStopping object at 0x00000281BF12DF40>, <keras.callbacks.LambdaCallback object at 0x00000281BF13F100>], epochs=250, model=<function create_model at 0x00000281C1861DC0>, model__activation1='tanh', model__activation2='tanh', model__activation3='relu', model__activation4='sel...te=0.0006832900387850664, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x00000283186EFA90>, <tensorflow_addons.metrics.r_square.RSquare object at 0x00000283186DD700>], model__optim=<class 'keras.optimizer_v2.rmsprop.RMSprop'>, print_summary=True, validation_split=0.2233573353320119, verbose=0),
                    clip_y=125, include_settings=True)

In [86]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_65 (Dense)            (None, 486)               12150     
                                                                 
 dropout_46 (Dropout)        (None, 486)               0         
                                                                 
 dense_66 (Dense)            (None, 470)               228890    
                                                                 
 dropout_47 (Dropout)        (None, 470)               0         
                                                                 
 dense_67 (Dense)            (None, 244)               114924    
                                                                 
 dropout_48 (Dropout)        (None, 244)               0         
                                                                 
 dense_68 (Dense)            (None, 123)             

In [87]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.701,RMSE=-23.510
Finished: 2022-11-03 09:45:54.879696


## PolyFeatures + Linear RUL

Score: 0.5298866447847905    
Test: 0.
```
('basemodel__batch_size', 370),
('basemodel__epochs', 33),
('basemodel__model__activation1', 'elu'),
('basemodel__model__activation2', 'sigmoid'),
('basemodel__model__activation3', 'sigmoid'),
('basemodel__model__activation4', 'elu'),
('basemodel__model__dropout1', 0.6870110349714994),
('basemodel__model__dropout2', 0.5029302361015952),
('basemodel__model__dropout3', 0.5605207131815848),
('basemodel__model__dropout4', 0.6403930728651405),
('basemodel__model__layer1', 428),
('basemodel__model__layer2', 483),
('basemodel__model__layer3', 286),
('basemodel__model__layer4', 94),
('basemodel__model__learning_rate', 0.0032976496820942666),
('basemodel__model__optim', keras.optimizer_v2.rmsprop.RMSprop),
('basemodel__validation_split', 0.12379763238695841),
('include_settings', True),
('poly_degree', 3),
('scaler', StandardScaler())
```


In [88]:
CLIP=-1

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=3,
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=370,
                           epochs=33,
                           model__activation1='elu',
                           model__activation2='sigmoid',
                           model__activation3='sigmoid',
                           model__activation4='elu',
                           model__dropout1=0.6870110349714994, 
                           model__dropout2=0.5029302361015952, 
                           model__dropout3=0.5605207131815848, 
                           model__dropout4=0.6403930728651405, 
                           model__layer1=428, 
                           model__layer2=483, 
                           model__layer3=286, 
                           model__layer4=94, 
                           model__learning_rate=0.0032976496820942666,
                           model__optim=RMSprop,
                           validation_split=0.12379763238695841, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=370, callbacks=[<keras.callbacks.EarlyStopping object at 0x00000281BF12DF40>, <keras.callbacks.LambdaCallback object at 0x00000281BF13F100>], epochs=33, model=<function create_model at 0x00000281C1861DC0>, model__activation1='elu', model__activation2='sigmoid', model__activation3='sigmoid', model__activation4=....0032976496820942666, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x000002831EABEA30>, <tensorflow_addons.metrics.r_square.RSquare object at 0x00000282F49C3520>], model__optim=<class 'keras.optimizer_v2.rmsprop.RMSprop'>, print_summary=True, validation_split=0.12379763238695841, verbose=0),
                    include_settings=True, poly_degree=3)

In [89]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_70 (Dense)            (None, 428)               1251900   
                                                                 
 dropout_50 (Dropout)        (None, 428)               0         
                                                                 
 dense_71 (Dense)            (None, 483)               207207    
                                                                 
 dropout_51 (Dropout)        (None, 483)               0         
                                                                 
 dense_72 (Dense)            (None, 286)               138424    
                                                                 
 dropout_52 (Dropout)        (None, 286)               0         
                                                                 
 dense_73 (Dense)            (None, 94)              

In [90]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.117,RMSE=-51.246
Finished: 2022-11-03 09:48:21.557574


## PolyFeatures + Non-Linear RUL


Score: 0.7547335898188002  
Test: 0.
```
('basemodel__batch_size', 33),
('basemodel__epochs', 35),
('basemodel__model__activation1', 'tanh'),
('basemodel__model__activation2', 'selu'),
('basemodel__model__activation3', 'selu'),
('basemodel__model__activation4', 'selu'),
('basemodel__model__dropout1', 0.35902813826517177),
('basemodel__model__dropout2', 0.10018359566403763),
('basemodel__model__dropout3', 0.3129879941596555),
('basemodel__model__dropout4', 0.3715931098343138),
('basemodel__model__layer1', 24),
('basemodel__model__layer2', 209),
('basemodel__model__layer3', 95),
('basemodel__model__layer4', 512),
('basemodel__model__learning_rate', 0.00029766030712635185),
('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.5105596671166837),
('clip_y', 127),
('include_settings', True),
('poly_degree', 2),
('scaler', StandardScaler())
```

In [91]:
CLIP=127

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=2,
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=33,
                           epochs=35,
                           model__activation1='tanh',
                           model__activation2='selu',
                           model__activation3='selu',
                           model__activation4='selu',
                           model__dropout1=0.35902813826517177, 
                           model__dropout2=0.10018359566403763, 
                           model__dropout3=0.3129879941596555, 
                           model__dropout4=0.3715931098343138, 
                           model__layer1=24, 
                           model__layer2=209, 
                           model__layer3=95, 
                           model__layer4=512, 
                           model__learning_rate=0.00029766030712635185,
                           model__optim=Adam,
                           validation_split=0.5105596671166837, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=33, callbacks=[<keras.callbacks.EarlyStopping object at 0x00000281BF12DF40>, <keras.callbacks.LambdaCallback object at 0x00000281BF13F100>], epochs=35, model=<function create_model at 0x00000281C1861DC0>, model__activation1='tanh', model__activation2='selu', model__activation3='selu', model__activation4='selu'...9766030712635185, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x00000283230FC0D0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x00000283186BD700>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.5105596671166837, verbose=0),
                    clip_y=127, include_settings=True, poly_degree=2)

In [92]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_75 (Dense)            (None, 24)                7800      
                                                                 
 dropout_54 (Dropout)        (None, 24)                0         
                                                                 
 dense_76 (Dense)            (None, 209)               5225      
                                                                 
 dropout_55 (Dropout)        (None, 209)               0         
                                                                 
 dense_77 (Dense)            (None, 95)                19950     
                                                                 
 dropout_56 (Dropout)        (None, 95)                0         
                                                                 
 dense_78 (Dense)            (None, 512)             

In [93]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.718,RMSE=-23.119
Finished: 2022-11-03 09:53:49.168980
